<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Attention_Networks_(GATs)_with_DGL_(Deep_Graph_Library).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dgl

In [ ]:
pip install torchdata

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import GATConv
from dgl.data import CoraGraphDataset

# Define the GAT model
class GAT(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, num_heads):
        super(GAT, self).__init__()
        self.layer1 = GATConv(in_feats, hidden_feats, num_heads)
        self.layer2 = GATConv(hidden_feats * num_heads, out_feats, num_heads)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = F.elu(h.flatten(1))
        h = self.layer2(g, h)
        return F.log_softmax(h.mean(1), dim=1)

# Load the dataset
dataset = CoraGraphDataset()
g = dataset[0]
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
test_mask = g.ndata['test_mask']

# Initialize the model, optimizer, and criterion
model = GAT(g.ndata['feat'].shape[1], 8, dataset.num_classes, 8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

# Training loop
model.train()
for epoch in range(200):
    logits = model(g, features)
    loss = F.nll_loss(logits[train_mask], labels[train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')